<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Library" data-toc-modified-id="Import-Library-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Library</a></span></li><li><span><a href="#Convert-Ndarray-to-Base64-String" data-toc-modified-id="Convert-Ndarray-to-Base64-String-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Convert Ndarray to Base64 String</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Filter-Data-Without-Labels" data-toc-modified-id="Filter-Data-Without-Labels-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filter Data Without Labels</a></span></li><li><span><a href="#Save-Results" data-toc-modified-id="Save-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save Results</a></span></li></ul></div>

## Import Library

In [4]:
import pandas as pd
import cv2
import base64
import time

## Convert Ndarray to Base64 String

In [5]:
def matrixToBase64(m):
    retval, buffer = cv2.imencode('.png', m)
    pic_str = base64.b64encode(buffer).decode()
    return pic_str

## Load Data

In [ ]:
t1 = time.time()
df=pd.read_pickle("/home/pengtan/Projects/python/image-classification-tutorial/data/wafer-map-pkl/LSWMD.pkl")
df.info()
elapsed = time.time() - t1
print("Loading Used time is " + str(elapsed))

df['failureNum']=df.failureType
df['trainTestNum']=df.trianTestLabel
mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,
              'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8}
mapping_traintest={'Training':0,'Test':1}
df=df.replace({'failureNum':mapping_type, 'trainTestNum':mapping_traintest})
t2 = time.time()
df['imgStr'] = [matrixToBase64(m) for m in df['waferMap']]
elapsed = time.time() - t2
print("Conversion used time is " + str(elapsed))
df['rowkey'] = df.index

## Filter Data Without Labels

In [6]:
df_withlabel = df[(df['failureNum']>=0) & (df['failureNum']<=8)]
df_withlabel =df_withlabel.reset_index()
df_withpattern = df[(df['failureNum']>=0) & (df['failureNum']<=7)]
df_withpattern = df_withpattern.reset_index()
df_nonpattern = df[(df['failureNum']==8)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
waferMap          811457 non-null object
dieSize           811457 non-null float64
lotName           811457 non-null object
waferIndex        811457 non-null float64
trianTestLabel    811457 non-null object
failureType       811457 non-null object
dtypes: float64(2), object(4)
memory usage: 37.1+ MB
Loading Used time is 125.80574345588684


/home/pengtan/Apps/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/internals/managers.py:1937: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


Conversion used time is 44.977057218551636


## Save Results

In [7]:
df_withlabel[['rowkey', 'lotName', 'waferIndex', 'dieSize', 'imgStr', 'failureNum', 'trainTestNum']].to_csv("/home/pengtan/Projects/python/image-classification-tutorial/data/wafer-map-pkl/waferMap.csv", header=True, index=False)